In [2]:
from google.colab import files
uploades = files.upload()

Saving Total_Data.xlsx to Total_Data.xlsx


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.utils import shuffle

In [4]:
df = pd.read_excel('Total_Data.xlsx') 

In [5]:
# Gives us the dataframe of team a_vs_team b for all past years
def a_vs_b(a,b):
    options = [a,b]
    a_b = df[df['Home Team'].isin(options)]
    a_b = a_b[a_b['Away Team'].isin(options)]
    a_b = pd.DataFrame(a_b)

    # Home Team(a) value is 1 and away team(b) is 0
    a_b.loc[a_b['Home Team'] == a,'Home Team'] = 1 
    a_b.loc[a_b['Away Team'] == a,'Away Team'] = 1
    a_b.loc[a_b['Home Team'] == b,'Home Team'] = 0
    a_b.loc[a_b['Away Team'] == b,'Away Team'] = 0
    return a_b

# Gives us the dataframe of team_vs_all for all past years
def team_vs_all(team):
    options = [team]
    team_all = df[df['Home Team'].isin(options) | df['Away Team'].isin(options)]

    #If team a wins then Win_Value = 1, if loses then 0, if draw then 0.5  
    team_all = pd.DataFrame(team_all)
    team_all.loc[((team_all['Home Team'] == team) & (team_all['FTR'] == 'H')) | ((team_all['Away Team'] == team) & (team_all['FTR'] == 'A')), 'Win_Value'] = 1
    team_all.loc[((team_all['Home Team'] == team) & (team_all['FTR'] == 'A')) | ((team_all['Away Team'] == team) & (team_all['FTR'] == 'H')), 'Win_Value'] = 0
    team_all = team_all.fillna(0.5)
    return team_all

# Gives us the dataframe of all the home matches of a team
def home_matches(team):
    options = [team]
    home_all = df[df['Home Team'].isin(options)]
    home_all = pd.DataFrame(home_all)
    return home_all

# Gives us the dataframe of all the away matches of a team
def away_matches(team):
    options = [team]
    away_all = df[df['Away Team'].isin(options)]
    away_all = pd.DataFrame(away_all) 
    return away_all

# Runs different classification algorithm and uses best one to classify the number of goals scored by a team
def goals(X,y,x,a):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

    knn = KNeighborsClassifier(n_neighbors = 5)
    knn.fit(X_train, y_train)
    ans1 = knn.score(X_test,y_test)
    p1 = knn.predict(X_test)
    
    clf = LinearSVC().fit(X_train, y_train)
    ans2 = clf.score(X_test,y_test)
    p2 = clf.predict(X_test)

    rbf = SVC(kernel = 'rbf', gamma = 0.1).fit(X_train, y_train)
    ans3 = rbf.score(X_test,y_test)
    p3 = rbf.predict(X_test)

    X, y = shuffle(X, y, random_state=0)
    if ((ans3>ans1) and (ans3>ans2)):
        rbf = SVC(kernel = 'rbf', gamma = 0.1).fit(X, y)
        print("Test accuracy rbf:", ans3)
        ans = rbf.predict(x)
        print("Prediction : ", p3)
    elif ans1>ans2:
        knn.fit(X, y)
        print("Test accuracy knn:", ans1)
        ans = knn.predict(x)
        print("Prediction : ", p1)
    else:
        clf = LinearSVC().fit(X, y)
        print("Test accuracy linearSVC:", ans2)
        ans = clf.predict(x)
        print("Prediction : ", p2)
    print("Actual     : ", y_test)
    print(a, "will score", int(ans), "goals")

In [6]:
a = input("Enter Home Team : ")
b = input("Enter Away Team : ")

Enter Home Team : Chelsea
Enter Away Team : Man City


In [20]:
a_all = team_vs_all(a) # team a vs all other teams data
b_all = team_vs_all(b) # team b vs all other team data
a_b = a_vs_b(a,b) # team a vs team b data
a_home = home_matches(a) # team a all home matches
b_home = home_matches(b) # team b all home matches
a_away = away_matches(a) # team a all away matches
b_away = away_matches(b) # team b all away matches

Pre processing of the data to extract new features 

In [21]:
#Foul values of both the teams
a_away['a_f'] = a_away['AF']
a_home['a_f'] = a_home['HF']
b_away['b_f'] = b_away['AF']
b_home['b_f'] = b_home['HF']
a_b = pd.merge(a_b,a_home[['Match_Id','a_f']], on='Match_Id',how = 'left')
a_b.loc[a_b.Match_Id.isin(a_away.Match_Id), ['a_f']] = a_away.loc[a_away.Match_Id.isin(a_b.Match_Id),['a_f']].values
a_b = pd.merge(a_b,b_home[['Match_Id','b_f']], on='Match_Id',how = 'left')
a_b.loc[a_b.Match_Id.isin(b_away.Match_Id), ['b_f']] = b_away.loc[b_away.Match_Id.isin(a_b.Match_Id),['b_f']].values
a_all = pd.merge(a_all,a_home[['Match_Id','a_f']], on='Match_Id',how = 'left')
a_all.loc[a_all.Match_Id.isin(a_away.Match_Id), ['a_f']] = a_away.loc[a_away.Match_Id.isin(a_all.Match_Id),['a_f']].values
b_all = pd.merge(b_all,b_home[['Match_Id','b_f']], on='Match_Id',how = 'left')
b_all.loc[b_all.Match_Id.isin(b_away.Match_Id), ['b_f']] = b_away.loc[b_away.Match_Id.isin(b_all.Match_Id),['b_f']].values

#Goals scored values of both the teams
a_away['a_goals'] = a_away['FTAG']
a_home['a_goals'] = a_home['FTHG']
b_away['b_goals'] = b_away['FTAG']
b_home['b_goals'] = b_home['FTHG']
a_b = pd.merge(a_b,a_home[['Match_Id','a_goals']], on='Match_Id',how = 'left')
a_b.loc[a_b.Match_Id.isin(a_away.Match_Id), ['a_goals']] = a_away.loc[a_away.Match_Id.isin(a_b.Match_Id),['a_goals']].values
a_b = pd.merge(a_b,b_home[['Match_Id','b_goals']], on='Match_Id',how = 'left')
a_b.loc[a_b.Match_Id.isin(b_away.Match_Id), ['b_goals']] = b_away.loc[b_away.Match_Id.isin(a_b.Match_Id),['b_goals']].values
a_all = pd.merge(a_all,a_home[['Match_Id','a_goals']], on='Match_Id',how = 'left')
a_all.loc[a_all.Match_Id.isin(a_away.Match_Id), ['a_goals']] = a_away.loc[a_away.Match_Id.isin(a_all.Match_Id),['a_goals']].values
b_all = pd.merge(b_all,b_home[['Match_Id','b_goals']], on='Match_Id',how = 'left')
b_all.loc[b_all.Match_Id.isin(b_away.Match_Id), ['b_goals']] = b_away.loc[b_away.Match_Id.isin(b_all.Match_Id),['b_goals']].values

#Number of yellow card values of both the teams
a_away['a_y'] = a_away['AY']
a_home['a_y'] = a_home['HY']
b_away['b_y'] = b_away['AY']
b_home['b_y'] = b_home['HY']
a_b = pd.merge(a_b,a_home[['Match_Id','a_y']], on='Match_Id',how = 'left')
a_b.loc[a_b.Match_Id.isin(a_away.Match_Id), ['a_y']] = a_away.loc[a_away.Match_Id.isin(a_b.Match_Id),['a_y']].values
a_b = pd.merge(a_b,b_home[['Match_Id','b_y']], on='Match_Id',how = 'left')
a_b.loc[a_b.Match_Id.isin(b_away.Match_Id), ['b_y']] = b_away.loc[b_away.Match_Id.isin(a_b.Match_Id),['b_y']].values
a_all = pd.merge(a_all,a_home[['Match_Id','a_y']], on='Match_Id',how = 'left')
a_all.loc[a_all.Match_Id.isin(a_away.Match_Id), ['a_y']] = a_away.loc[a_away.Match_Id.isin(a_all.Match_Id),['a_y']].values
b_all = pd.merge(b_all,b_home[['Match_Id','b_y']], on='Match_Id',how = 'left')
b_all.loc[b_all.Match_Id.isin(b_away.Match_Id), ['b_y']] = b_away.loc[b_away.Match_Id.isin(b_all.Match_Id),['b_y']].values

#Number of red card values of both the teams
a_away['a_r'] = a_away['AR']
a_home['a_r'] = a_home['HR']
b_away['b_r'] = b_away['AR']
b_home['b_r'] = b_home['HR']
a_b = pd.merge(a_b,a_home[['Match_Id','a_r']], on='Match_Id',how = 'left')
a_b.loc[a_b.Match_Id.isin(a_away.Match_Id), ['a_r']] = a_away.loc[a_away.Match_Id.isin(a_b.Match_Id),['a_r']].values
a_b = pd.merge(a_b,b_home[['Match_Id','b_r']], on='Match_Id',how = 'left')
a_b.loc[a_b.Match_Id.isin(b_away.Match_Id), ['b_r']] = b_away.loc[b_away.Match_Id.isin(a_b.Match_Id),['b_r']].values
a_all = pd.merge(a_all,a_home[['Match_Id','a_r']], on='Match_Id',how = 'left')
a_all.loc[a_all.Match_Id.isin(a_away.Match_Id), ['a_r']] = a_away.loc[a_away.Match_Id.isin(a_all.Match_Id),['a_r']].values
b_all = pd.merge(b_all,b_home[['Match_Id','b_r']], on='Match_Id',how = 'left')
b_all.loc[b_all.Match_Id.isin(b_away.Match_Id), ['b_r']] = b_away.loc[b_away.Match_Id.isin(b_all.Match_Id),['b_r']].values

Processing of data to get required features for prediction of number of goals scored

In [22]:
# HSTP and ASTP average of both teams for past 5 matches
values = []
list_values = []
sum = 0

for value in b_all['HSTP']:
    values.append(value)
    
for i in range(len(values)):
    if i<5:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-5]
        prob = sum / 5
    list_values.append(prob) 

b_all["HSTP_avg"] = list_values
b_all['HSTP_avg'] = np.roll(b_all['HSTP_avg'], shift=1)

values = []
list_values = []
sum = 0

for value in b_all['ASTP']:
    values.append(value)
    
for i in range(len(values)):
    if i<5:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-5]
        prob = sum / 5
    list_values.append(prob) 

b_all["ASTP_avg"] = list_values
b_all['ASTP_avg'] = np.roll(b_all['ASTP_avg'], shift=1)

values = []
list_values = []
sum = 0

for value in a_all['ASTP']:
    values.append(value)
    
for i in range(len(values)):
    if i<5:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-5]
        prob = sum / 5
    list_values.append(prob) 

a_all["ASTP_avg"] = list_values
a_all['ASTP_avg'] = np.roll(a_all['ASTP_avg'], shift=1)

values = []
list_values = []
sum = 0

for value in a_all['HSTP']:
    values.append(value)
    
for i in range(len(values)):
    if i<5:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-5]
        prob = sum / 5
    list_values.append(prob) 

a_all["HSTP_avg"] = list_values
a_all['HSTP_avg'] = np.roll(a_all['HSTP_avg'], shift=1)

values = []
list_values = []
sum = 0

# Goals scored average of both teams for past 5 matches
for value in a_all['a_goals']:
    values.append(value)
    
for i in range(len(values)):
    if i<5:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-5]
        prob = sum / 5
    list_values.append(prob) 

a_all["a_goals_5"] = list_values
a_all['a_goals_5'] = np.roll(a_all['a_goals_5'], shift=1)

values = []
list_values = []
sum = 0

for value in b_all['b_goals']:
    values.append(value)
    
for i in range(len(values)):
    if i<5:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-5]
        prob = sum / 5
    list_values.append(prob) 

b_all["b_goals_5"] = list_values
b_all['b_goals_5'] = np.roll(b_all['b_goals_5'], shift=1)

values = []
list_values = []
sum = 0

# Fouls scored average of both teams for past 5 matches
for value in a_all['a_f']:
    values.append(value)
    
for i in range(len(values)):
    if i<5:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-5]
        prob = sum / 5
    list_values.append(prob) 

a_all["a_f5"] = list_values
a_all['a_f5'] = np.roll(a_all['a_f5'], shift=1)

values = []
list_values = []
sum = 0

for value in b_all['b_f']:
    values.append(value)
    
for i in range(len(values)):
    if i<5:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-5]
        prob = sum / 5
    list_values.append(prob) 

b_all["b_f5"] = list_values
b_all['b_f5'] = np.roll(b_all['b_f5'], shift=1)

values = []
list_values = []
sum = 0

# Yellow cards average of both teams for past 5 matches
for value in a_all['a_y']:
    values.append(value)
    
for i in range(len(values)):
    if i<5:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-5]
        prob = sum / 5
    list_values.append(prob) 

a_all["a_y5"] = list_values
a_all['a_y5'] = np.roll(a_all['a_y5'], shift=1)

values = []
list_values = []
sum = 0

for value in b_all['b_y']:
    values.append(value)
    
for i in range(len(values)):
    if i<5:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-5]
        prob = sum / 5
    list_values.append(prob) 

b_all["b_y5"] = list_values
b_all['b_y5'] = np.roll(b_all['b_y5'], shift=1)

values = []
list_values = []
sum = 0

# Red Cards average of both teams for past 5 matches
for value in a_all['a_r']:
    values.append(value)
    
for i in range(len(values)):
    if i<5:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-5]
        prob = sum / 5
    list_values.append(prob) 

a_all["a_r5"] = list_values
a_all['a_r5'] = np.roll(a_all['a_r5'], shift=1)

values = []
list_values = []
sum = 0

for value in b_all['b_r']:
    values.append(value)
    
for i in range(len(values)):
    if i<5:
        sum = sum + values[i]
        prob = sum / (i+1)
    else:
        sum = sum + values[i] - values[i-5]
        prob = sum / 5
    list_values.append(prob) 

b_all["b_r5"] = list_values
b_all['b_r5'] = np.roll(b_all['b_r5'], shift=1)

Merging of different features into a single dataset

In [10]:
data = pd.merge(a_b,a_all[['Match_Id','a_f5']], on='Match_Id', how='left')
data = pd.merge(data,b_all[['Match_Id','b_f5']], on='Match_Id', how='left')
data = pd.merge(data,a_all[['Match_Id','a_y5']], on='Match_Id', how='left')
data = pd.merge(data,b_all[['Match_Id','b_y5']], on='Match_Id', how='left')
data = pd.merge(data,a_all[['Match_Id','a_r5']], on='Match_Id', how='left')
data = pd.merge(data,b_all[['Match_Id','b_r5']], on='Match_Id', how='left')
data = pd.merge(data,a_all[['Match_Id','a_goals_5']], on='Match_Id', how='left')
data = pd.merge(data,b_all[['Match_Id','b_goals_5']], on='Match_Id', how='left')

data = pd.merge(data,a_all[['Match_Id','ASTP_avg']], on='Match_Id',how = 'left')
data.loc[data.Match_Id.isin(b_all.Match_Id), ['ASTP_avg']] = b_all.loc[b_all.Match_Id.isin(data.Match_Id),['ASTP_avg']].values
data = pd.merge(data,a_all[['Match_Id','HSTP_avg']], on='Match_Id',how = 'left')
data.loc[data.Match_Id.isin(b_all.Match_Id), ['HSTP_avg']] = b_all.loc[b_all.Match_Id.isin(data.Match_Id),['HSTP_avg']].values

Data Manipulation
#####X = Input Features dataset 
#####y1 = Goals scored by team a
#####y2 - Goals scored by team b
#####x = Features on which we want outcome of prediction algorithm

In [11]:
columns = ['Home Team','Away Team','a_goals_5', 'b_goals_5', 'HSTP_avg', 'ASTP_avg', 'a_f5', 'b_f5']
x = data[columns]
x = x[0:1]
x['Home Team'] = 1
x['Away Team'] = 0
x['a_f5'] = a_all.iloc[0]['a_f5']
x['b_f5'] = b_all.iloc[0]['b_f5']
x['a_goals_5'] = a_all.iloc[0]['a_goals_5']
x['b_goals_5'] = b_all.iloc[0]['b_goals_5']
x['HSTP_avg'] = a_all.iloc[0]['HSTP_avg']
x['ASTP_avg'] = b_all.iloc[0]['ASTP_avg']

X = data[columns]
X = X[2:]

y1 = data[['a_goals']]
y1 = y1[2:]
y1 = y1.values.ravel()

y2 = data[['b_goals']]
y2 = y2[2:]
y2 = y2.values.ravel()

In [13]:
goals(X,y1,x,a)

Test accuracy linearSVC: 0.5
Prediction :  [2. 0. 0. 1. 0. 0. 0. 6. 1. 0.]
Actual     :  [2. 0. 1. 1. 0. 2. 2. 0. 1. 2.]
Chelsea will score 2 goals


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [14]:
goals(X,y2,x,b)

Test accuracy knn: 0.6
Prediction :  [0. 0. 0. 0. 0. 1. 1. 1. 1. 0.]
Actual     :  [4. 3. 2. 0. 0. 1. 0. 1. 1. 0.]
Man City will score 1 goals


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
